<a href="https://colab.research.google.com/github/ladsong/if697-2020.2-data-science/blob/projeto2/Projeto_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Preparando o ambiente

In [1]:
import pandas as pd
import numpy as np
import re
import gc
import warnings
import sys

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
# /content/gdrive/My Drive/Kaggle é o caminho onde o arquivo kaggle.json está presente do Google Drive
#Mudar o diretorio
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


##Carregando o dataset e processando os dados
No projeto 1, foi adicionado o dump dos dados para que possamos utiliza-los aqui. O objetivo desse projeto é predizer a popularidade de uma música através de suas caracteristicas informadas no subset.

In [4]:
df = pd.read_csv('project1_output.csv')

Observamos que o dataset é grande e assim decidimos utilizar um subset  com o objetivo de diminuir o tempo de experimentação

In [5]:
df = df[:5000]
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df.dtypes

year                   int64
acousticness         float64
artists_data          object
danceability         float64
duration_ms            int64
energy               float64
explicit               int64
id                    object
instrumentalness     float64
loudness             float64
name                  object
popularity           float64
release_date          object
speechiness          float64
tempo                float64
first_artist          object
artists_by_artist     object
first_genre           object
dtype: object

Tirando colunas desnecessárias, como `artist_data, id, name, release_date, first_artist, artists_by_artist` e `first_genre,` pois só queremos trabalhar com valores numéricos




In [6]:
df = df.select_dtypes(exclude=['object'])

In [7]:
df.columns

Index(['year', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'explicit', 'instrumentalness', 'loudness', 'popularity', 'speechiness',
       'tempo'],
      dtype='object')



###Escolher coluna para predição

In [8]:
target_col = df['popularity']

In [9]:
target_col

0       0.04
1       0.02
2       0.04
3       0.00
4       0.01
        ... 
4995    0.00
4996    0.00
4997    0.00
4998    0.00
4999    0.00
Name: popularity, Length: 5000, dtype: float64

In [10]:
df = df.drop(columns=['popularity'])

##Separando os dados em teste e predição
Para realizacão do treinamento, teste e vaidacão, iremos separa o subset em: 

*   3/5 dos dados para treinamento
*   1/5 dos dados para teste 
*   1/5 dos dados para validacão

Seguindo assim, uma base 60/20/20



In [11]:
def get_x_data():
    # input 
    train, val, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
    
    return train, val, test

In [12]:
def get_y_data():
    # output
    train_labels, val_labels, test_labels = (
        np.split(
            target_col, 
            [int(.6*len(target_col)), int(.8*len(target_col))])
    )
    
    return train_labels, val_labels, test_labels

##Escolher os 4 algoritmos para predição

Os algoritmos de predicão que iremos usar são:

*   Regressão Linear
*   Multilayer perceptron
*   Random forests
*   Gradient boost com lightgbm


In [13]:
!pip install mlflow --quiet

     |████████████████████████████████| 14.4 MB 63 kB/s 
     |████████████████████████████████| 56 kB 3.9 MB/s 
     |████████████████████████████████| 636 kB 53.4 MB/s 
     |████████████████████████████████| 1.1 MB 53.7 MB/s 
     |████████████████████████████████| 79 kB 7.5 MB/s 
     |████████████████████████████████| 170 kB 65.6 MB/s 
     |████████████████████████████████| 146 kB 69.7 MB/s 
     |████████████████████████████████| 75 kB 3.9 MB/s 
     |████████████████████████████████| 52 kB 1.1 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 


In [14]:
!pip install optuna

     |████████████████████████████████| 302 kB 14.1 MB/s 
     |████████████████████████████████| 80 kB 8.3 MB/s 
     |████████████████████████████████| 141 kB 84.4 MB/s 
     |████████████████████████████████| 49 kB 5.1 MB/s 
     |████████████████████████████████| 111 kB 97.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=a3755124bc14d89ea8f30cf7432824a38c329709fbce8e8bc3b21a1d5a467fb7
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [15]:
import mlflow
import mlflow.sklearn

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision
from tensorflow.keras.layers.experimental import preprocessing

import optuna

from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    confusion_matrix,
    classification_report,
    accuracy_score
)
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

import lightgbm
from lightgbm import LGBMRegressor

###Função de avaliação das metricas


In [16]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [17]:
mlflow.sklearn.autolog()
mlflow.tensorflow.autolog()
mlflow.lightgbm.autolog()

2021/08/17 01:51:42 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of lightgbm. If you encounter errors during autologging, try upgrading / downgrading lightgbm to a supported version, or try upgrading MLflow.


###Regressão Linear

In [20]:
def linear_regression(trial):
    train, test, val = get_x_data()
    train_labels, val_labels, test_labels = get_y_data()
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    with mlflow.start_run(run_name="Linear Regression"):
        reg = LinearRegression()
        reg.fit(train, train_labels)

        predictions = reg.predict(val)

        (rmse, mae, r2) = eval_metrics(val_labels, predictions)

        print("Modelo de regressão linear")
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("r2", r2)
        
        
        gc.collect()
        
        return rmse

In [21]:
study = optuna.create_study()
study.optimize(linear_regression, n_trials=1)

[I 2021-08-17 01:52:05,071] A new study created in memory with name: no-name-614d2d2b-0b88-42aa-970c-8ff6e73a0d1c


Modelo de regressão linear
  RMSE: 0.13007133766127812
  MAE: 0.08361231897096476
  R2: -0.05711240737137979


[I 2021-08-17 01:52:06,995] Trial 0 finished with value: 0.13007133766127812 and parameters: {}. Best is trial 0 with value: 0.13007133766127812.


###Multilayer Perceptron

In [22]:
def mlp(trial):
    train, test, val = get_x_data()
    train_labels, val_labels, test_labels = get_y_data()
    
    params = {
        "hidden_units": trial.suggest_int("hidden_units", 3, 15),
        "lr": trial.suggest_float("lr", 1e-5, 1e-3, log=True),
        "epochs": trial.suggest_int("epochs", 10, 50)
    }
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    with mlflow.start_run(run_name="MLP"):
        normalizer = preprocessing.Normalization(axis=-1)
        normalizer.adapt(np.array(train))
        
        mlp_model = tf.keras.Sequential([
            normalizer,
            layers.Dense(units=params["hidden_units"]),
            layers.Dense(units=params["hidden_units"]),
            layers.Dense(units=params["hidden_units"]),
            layers.Dense(units=1),
        ])

        mlp_model.summary()
        
        mlp_model.compile(
            optimizer=tf.optimizers.Adam(learning_rate=params["lr"]),
            loss='mean_squared_error'
        )

        history = mlp_model.fit(
            train, train_labels,
            validation_data=(test, test_labels),
            epochs=params["epochs"]
        )
        
        predictions = mlp_model.predict(val)

        (rmse, mae, r2) = eval_metrics(val_labels, predictions)

        print("MLP model")
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("r2", r2)
        mlflow.log_params(trial.params)
        mlflow.set_tags(
            {
                "estimator_name":"MultiLayerPerceptron",
                "estimator_class":"Keras"
            }
        )
        
        tf.keras.backend.clear_session()

        gc.collect()
        
        return rmse

In [23]:

study = optuna.create_study()
study.optimize(mlp, n_trials=10)

[I 2021-08-17 01:52:23,333] A new study created in memory with name: no-name-55685bdd-d895-499b-bda0-c45dd0c9ce38


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 10)                21        
_________________________________________________________________
dense (Dense)                (None, 5)                 55        
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 142
Trainable params: 121
Non-trainable params: 21
_________________________________________________________________
Epoch 1/19
94/94 [==============================] - 1s 3ms/step - loss: 0.4791 - val_loss: 0.5618
Epoch 2/19
94/94 [=======

[I 2021-08-17 01:52:31,138] Trial 0 finished with value: 0.49359967767518986 and parameters: {'hidden_units': 5, 'lr': 2.3814351954450575e-05, 'epochs': 19}. Best is trial 0 with value: 0.49359967767518986.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 10)                21        
_________________________________________________________________
dense (Dense)                (None, 3)                 33        
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 12        
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 12        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 4         
Total params: 82
Trainable params: 61
Non-trainable params: 21
_________________________________________________________________
Epoch 1/14
94/94 [==============================] - 1s 3ms/step - loss: 0.0779 - val_loss: 0.0922
Epoch 2/14
94/94 [=========

[I 2021-08-17 01:52:35,967] Trial 1 finished with value: 0.3198517923770368 and parameters: {'hidden_units': 3, 'lr': 1.0002275909239583e-05, 'epochs': 14}. Best is trial 1 with value: 0.3198517923770368.


...

[I 2021-08-17 01:53:23,536] Trial 7 finished with value: 0.19941255826246648 and parameters: {'hidden_units': 5, 'lr': 5.9367778049817834e-05, 'epochs': 29}. Best is trial 3 with value: 0.12930161105549232.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 10)                21        
_________________________________________________________________
dense (Dense)                (None, 13)                143       
_________________________________________________________________
dense_1 (Dense)              (None, 13)                182       
_________________________________________________________________
dense_2 (Dense)              (None, 13)                182       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 14        
Total params: 542
Trainable params: 521
Non-trainable params: 21
_________________________________________________________________
Epoch 1/18
94/94 [==============================] - 1s 3ms/step - loss: 0.2988 - val_loss: 0.0776
Epoch 2/18
94/94 [=======

[I 2021-08-17 01:53:29,450] Trial 8 finished with value: 0.12954275149138028 and parameters: {'hidden_units': 13, 'lr': 0.0002725172979947129, 'epochs': 18}. Best is trial 3 with value: 0.12930161105549232.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 10)                21        
_________________________________________________________________
dense (Dense)                (None, 10)                110       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 362
Trainable params: 341
Non-trainable params: 21
_________________________________________________________________
Epoch 1/37
94/94 [==============================] - 1s 3ms/step - loss: 1.3910 - val_loss: 1.2909
Epoch 2/37
94/94 [=======

[I 2021-08-17 01:53:41,708] Trial 9 finished with value: 0.1368383551760044 and parameters: {'hidden_units': 10, 'lr': 5.2491145582008663e-05, 'epochs': 37}. Best is trial 3 with value: 0.12930161105549232.


###Random Forest

In [24]:
def random_forest(trial):
    train, test, val = get_x_data()
    train_labels, val_labels, test_labels = get_y_data()
    
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 150),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 5),
    }
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    with mlflow.start_run(run_name="Random Forest"):
        rf = RandomForestRegressor(
            max_depth=params["max_depth"],
            n_estimators=params["n_estimators"],
            min_samples_split=params["min_samples_split"],
            random_state=0
        )
        rf.fit(train, train_labels)
        
        predictions = rf.predict(val)
        
        (rmse, mae, r2) = eval_metrics(val_labels, predictions)
        
        print("Random Forest model")
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)
        
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("r2", r2)
        mlflow.log_params(trial.params)
        
        gc.collect()
        
        return rmse

In [25]:
study = optuna.create_study()
study.optimize(random_forest, n_trials=10)

[I 2021-08-17 01:53:41,766] A new study created in memory with name: no-name-d93f78eb-af08-48d5-8188-642013a8feac


Random Forest model
  RMSE: 0.13084474164800078
  MAE: 0.08465437294847354
  R2: -0.06972095918238397


[I 2021-08-17 01:53:44,180] Trial 0 finished with value: 0.13084474164800078 and parameters: {'n_estimators': 137, 'max_depth': 7, 'min_samples_split': 5}. Best is trial 0 with value: 0.13084474164800078.


Random Forest model
  RMSE: 0.13100121422779862
  MAE: 0.08449373537016633
  R2: -0.0722809715929904


[I 2021-08-17 01:53:45,540] Trial 1 finished with value: 0.13100121422779862 and parameters: {'n_estimators': 106, 'max_depth': 3, 'min_samples_split': 3}. Best is trial 0 with value: 0.13084474164800078.


...

[I 2021-08-17 01:53:56,755] Trial 7 finished with value: 0.13139459834586267 and parameters: {'n_estimators': 150, 'max_depth': 9, 'min_samples_split': 3}. Best is trial 0 with value: 0.13084474164800078.


Random Forest model
  RMSE: 0.13126225526177487
  MAE: 0.08519574618498782
  R2: -0.07655861462737845


[I 2021-08-17 01:53:59,494] Trial 8 finished with value: 0.13126225526177487 and parameters: {'n_estimators': 126, 'max_depth': 10, 'min_samples_split': 4}. Best is trial 0 with value: 0.13084474164800078.


Random Forest model
  RMSE: 0.13102418006574745
  MAE: 0.08485249464800479
  R2: -0.07265696794925369


[I 2021-08-17 01:54:01,066] Trial 9 finished with value: 0.13102418006574745 and parameters: {'n_estimators': 67, 'max_depth': 8, 'min_samples_split': 5}. Best is trial 0 with value: 0.13084474164800078.


###Gradient boost com lightgbm

In [26]:
def gradient_boosting(trial):
    train, test, val = get_x_data()
    train_labels, val_labels, test_labels = get_y_data()
    
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 150),
        "num_leaves": trial.suggest_int("num_leaves", 25, 35),
        "max_depth": trial.suggest_int("max_depth", 3, 10)
    }
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    with mlflow.start_run(run_name="Gradient Boosting"):
        model = XGBRegressor(
            max_depth=params["max_depth"],
            n_estimators=params["n_estimators"],
        )
        model.fit(train, train_labels)
        
        predictions = model.predict(test)
        print('Prediction: %.3f' % predictions[0])
        
        (rmse, mae, r2) = eval_metrics(val_labels, predictions)

        print("LGBM model")
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log mlflow attributes for mlflow UI
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("r2", r2)
        mlflow.log_params(trial.params)
        mlflow.set_tags(
            {
                "estimator_class":"LightGBM",
                "estimator_name":"Gradient Boosting"
            }
        )
        mlflow.sklearn.log_model(model, "model")
        
        gc.collect()
        
        return rmse

In [27]:
study = optuna.create_study()
study.optimize(gradient_boosting, n_trials=10)

[I 2021-08-17 01:54:01,107] A new study created in memory with name: no-name-3378c745-db54-4415-949f-c658fb28c8c2


[01:54:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Prediction: 0.049
LGBM model
  RMSE: 0.1345920713821432
  MAE: 0.08818051055431367
  R2: -0.13187094308558778


[I 2021-08-17 01:54:02,025] Trial 0 finished with value: 0.1345920713821432 and parameters: {'n_estimators': 143, 'num_leaves': 27, 'max_depth': 5}. Best is trial 0 with value: 0.1345920713821432.


[01:54:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Prediction: 0.045
LGBM model
  RMSE: 0.13220877903262113
  MAE: 0.08500739752054215
  R2: -0.09214058443862028


[I 2021-08-17 01:54:02,731] Trial 1 finished with value: 0.13220877903262113 and parameters: {'n_estimators': 94, 'num_leaves': 33, 'max_depth': 4}. Best is trial 1 with value: 0.13220877903262113.


...

[I 2021-08-17 01:54:09,034] Trial 7 finished with value: 0.13673111845888022 and parameters: {'n_estimators': 100, 'num_leaves': 28, 'max_depth': 8}. Best is trial 2 with value: 0.1318033902047479.


[01:54:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Prediction: 0.047
LGBM model
  RMSE: 0.13184100254033523
  MAE: 0.08483837148308754
  R2: -0.08607283364196094


[I 2021-08-17 01:54:09,750] Trial 8 finished with value: 0.13184100254033523 and parameters: {'n_estimators': 74, 'num_leaves': 29, 'max_depth': 4}. Best is trial 2 with value: 0.1318033902047479.


[01:54:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Prediction: 0.054
LGBM model
  RMSE: 0.1336722088954913
  MAE: 0.08643259169697762
  R2: -0.1164523866494318


[I 2021-08-17 01:54:10,652] Trial 9 finished with value: 0.1336722088954913 and parameters: {'n_estimators': 147, 'num_leaves': 30, 'max_depth': 3}. Best is trial 2 with value: 0.1318033902047479.


# Análise de melhor modelo de predicão

In [31]:
!pip install pyngrok --quiet

import mlflow

with mlflow.start_run(run_name="MLflow on Colab"):
  mlflow.log_metric("m1", 2.0)
  mlflow.log_param("p1", "mlflow-colab")

# run tracking UI in the background
get_ipython().system_raw("mlflow ui --port 5000 &") # run tracking UI in the background


# create remote tunnel using ngrok.com to allow local port access
# borrowed from https://colab.research.google.com/github/alfozan/MLflow-GBRT-demo/blob/master/MLflow-GBRT-demo.ipynb#scrollTo=4h3bKHMYUIG6

from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = ""
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://3633a99cf690.ngrok.io
